In [1]:
import re #正则biaodas
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [2]:
#用pandas读入数据
train = pd.read_csv("./dataset/labeledTrainData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)
test = pd.read_csv("./dataset/testData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)

In [3]:
def review_to_wordlist(review,remove_stopwords=False):
    """
function to make review into wordlist:
1.去网页标签
2.去掉数字
3.以空格为分隔符分割句子
4.option：如果发现有字符在停用字符中，删除掉
"""
    #1.去网页标签
    cleaned_review = BeautifulSoup(review).get_text()
    #2.去掉数字
    cleaned_review = re.sub("[^a-zA-Z]"," ",cleaned_review)
    #3.split review into wordlist
    words = cleaned_review.lower().split()
    #4.stop_words if ask
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        words = [word for word in words if word not in stop_words]
    return words

In [4]:
review_to_wordlist(train["review"][0],remove_stopwords=True)

/home/alex/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/alex/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [5]:
#将train/test中的review转换成wordlist
train_data = []
for i in range(len(train["review"])):
    if (i+1)%5000 == 0:
        print("train review :{} / {}".format(i+1,len(train["review"])))
    train_data.append(" ".join(review_to_wordlist(train["review"][i])))
print("Done with train : reviews to word list")

test_data = []
for i in range(len(test["review"])):
    if (i+1)%5000 == 0:
        print("test review :{} / {}".format(i+1,len(test["review"])))
    train_data.append(" ".join(review_to_wordlist(test["review"][i])))
print("Done with test : reviews to word list")

#training label y
y_train = train["sentiment"]

/home/alex/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/alex/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


train review :5000 / 25000
train review :10000 / 25000
train review :15000 / 25000
train review :20000 / 25000
train review :25000 / 25000
Done with train : reviews to word list
test review :5000 / 25000
test review :10000 / 25000
test review :15000 / 25000
test review :20000 / 25000
test review :25000 / 25000
Done with test : reviews to word list


Use TF-IDF to generate features 

-About TF-IDF : https://blog.csdn.net/zrc199021/article/details/53728499

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV

In [7]:
#初始化TF-IDF对象，去停用词。加二元语言模型
tfv = TFIV(min_df=3,
           max_features=None,
           strip_accents='unicode',
           analyzer='word',
           token_pattern=r'\w{1,}',
           ngram_range=(1, 2),
           use_idf=1,smooth_idf=1,
           sublinear_tf=1,
           stop_words = 'english')

#生成tf-idf feature
All_reviews = train_data+test_data
tfv.fit(All_reviews)
All_review_tfidf = tfv.transform(All_reviews)


In [8]:
train_tfidf = All_review_tfidf[:len(train["review"])]
test_tfidf = All_review_tfidf[len(train["review"]):]


In [9]:
#构建朴素贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB as MNB

model_NB = MNB()
model_NB.fit(train_tfidf, y_train) #特征数据直接灌进来
MNB(alpha=1.0, class_prior=None, fit_prior=True)
from sklearn.cross_validation import cross_val_score
import numpy as np
print ("多项式贝叶斯分类器20折交叉验证得分: ", np.mean(cross_val_score(model_NB, train_tfidf, y_train, cv=20, scoring='roc_auc')))


/home/alex/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


多项式贝叶斯分类器20折交叉验证得分:  0.94963712
